# Practica uno

Para la realización de esta práctica solo se usará la librería de numpy.

In [1]:
import numpy as np

## Ejercicio 1



In [ ]:
class CodificadorEtiqueta:
  def __init__(self):
    pass

  def 